In [11]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
import requests

In [12]:
import dotenv

dotenv.load_dotenv()

True

In [13]:
# Keys
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [14]:
# Init
pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                         environment=PINECONE_ENVIRONMENT)

embeddings = CohereEmbeddings(model="multilingual-22-12")
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [15]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)


In [16]:
chain.invoke("what is film noir?")

'Based on the provided context, which is empty, I cannot give you a tailored answer. However, I can provide a general definition.\n\nFilm noir is a cinematic term used primarily to describe stylish Hollywood crime dramas, particularly those that emphasize cynical attitudes and sexual motivations. The term is French for "black film" (literal) or "dark film" (closer meaning), and it was first applied to Hollywood films by French critics who noticed the trend of how dark, downbeat, and black the looks and themes were of many American crime and detective films released in France following World War II.\n\nFilm noir of the 1940s and 1950s is associated with a low-key, black-and-white visual style that has roots in German Expressionist cinematography. Common themes in film noir include murder, crime, mystery, and the femme fatale. Many film noirs tend to revolve around an existentially disillusioned male protagonist - often a detective or an otherwise tough character who encounters a seducti

In [17]:
def fetch_wikipedia_page(id):
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"
    response = requests.get(url)
    data = response.json()
    page_content = list(data['query']['pages'].values())[0]['extract']
    return page_content

def fetch_url(x):
    urls = [doc.metadata['url'] for doc in x['context']]
    ids = [url.split('=')[-1] for url in urls]
    # First 32k tokens
    contents = [fetch_wikipedia_page(id)[:32000] for id in ids]    
    return {"context": contents, "question": x["question"]}

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | RunnableLambda(fetch_url)  
    | prompt
    | model
    | StrOutputParser()
)


In [18]:
chain.invoke("what is film noir?")

'Film noir is a cinematic term used primarily to describe stylish Hollywood crime dramas, particularly those that emphasize cynical attitudes and sexual motivations. The term is French for "black film" and was first applied by French critics to American thrillers from the 1940s and 1950s that were marked by a mood of pessimism, fatalism, and menace. Classic film noir developed during and after World War II, taking advantage of the post-war ambience of anxiety, disillusionment, and a sense of foreboding.\n\nFilm noir is known for its black-and-white visual style, which has roots in German Expressionist cinematography. It often involves a crime story, with the central figure being a hard-boiled detective or a flawed protagonist who operates in a corrupt legal or political system. Common themes include the nature of evil, the menace of urban life, existential despair, and the questionable moral choices of the characters. The narrative is frequently non-linear, with the use of flashbacks a